# Example usage notebook of the DD-GAN repo

In [1]:
import time
import numpy as np
import sklearn.preprocessing
import tensorflow as tf

After installation of the ddgan package run the following line to import it

In [2]:
from ddgan import GAN
from ddgan import *

In [3]:
kwargs = {
    "nsteps" : 5,
    "ndims" : 5,
    "lmbda" : 10,
    "n_critic" : 5,
    "batches" : 10,
    "batch_size" : 20,
    "seed" : 143
}

gan = GAN(**kwargs)
gan.setup()
set_seed(gan.seed)

ndims_latent_input = 5

#csv_data = np.load('../data/processed/pod_coeffs_field_Velocity_onegrid.npy', )
csv_data = np.loadtxt('../data/processed/POD_coeffs_1_204_orig.csv', delimiter=',')
csv_data = np.float32(csv_data)
#csv_data = csv_data[0,:,:]ææææææææææáððððáþÞÆ
                    

print('type and shape (nPOD by nTrain) of POD coeffs from csv file', type(csv_data), csv_data.shape, csv_data.dtype)


looking for previous saved models
making new generator and critic
type and shape (nPOD by nTrain) of POD coeffs from csv file <class 'numpy.ndarray'> (5, 204) float32


In [4]:
nTrain = csv_data.shape[1]
nPOD = csv_data.shape[0]

csv_data = csv_data.T # nTrain by nPOD


In [5]:
scaling = sklearn.preprocessing.MinMaxScaler(feature_range=[-1,1])
csv_data = scaling.fit_transform(csv_data)

for icol in range(csv_data.shape[1]):
    print('min and max of col, ', icol ,' of csv_data:', np.min(csv_data[:,icol]), np.max(csv_data[:,icol]) )

min and max of col,  0  of csv_data: -1.0 1.0
min and max of col,  1  of csv_data: -1.0 1.0000001
min and max of col,  2  of csv_data: -1.0 1.0
min and max of col,  3  of csv_data: -1.0 1.0
min and max of col,  4  of csv_data: -1.0 1.0


In [6]:
nsteps = 5
t_begin = 0
t_end = nTrain - nsteps + 1
training_data = np.zeros((t_end, nPOD * nsteps), dtype=np.float32) # nTrain by nsteps*nPOD # 'float32' or np.float32

input_to_GAN = tf.random.normal([training_data.shape[0], ndims_latent_input])
input_to_GAN = input_to_GAN.numpy()

for step in range(nsteps):
    training_data[:,step*nPOD:(step+1)*nPOD] = csv_data[t_begin+step : t_end+step,:]
print('Shape of training data for the GAN', training_data.shape, training_data.dtype)

Shape of training data for the GAN (200, 25) float32


In [7]:
t0 = time.time()
gan.learn_hypersurface_from_POD_coeffs(nPOD, input_to_GAN, training_data, ndims_latent_input)
t_train = time.time() - t0

beginning training
epoch: 0
gen loss:  0.00044128043 d loss:  9.975678 w_loss:  0.00051889766
epoch: 1
gen loss:  0.0014283005 d loss:  9.944265 w_loss:  0.0013156604
epoch: 2
gen loss:  0.0023684455 d loss:  9.915084 w_loss:  0.0020202943
epoch: 3
gen loss:  0.0036040687 d loss:  9.864314 w_loss:  0.0030076883
epoch: 4
gen loss:  0.0047299913 d loss:  9.823099 w_loss:  0.0041037756
epoch: 5
gen loss:  0.0060255853 d loss:  9.740541 w_loss:  0.0058216257
epoch: 6
gen loss:  0.007647969 d loss:  9.670988 w_loss:  0.0073640244
epoch: 7
gen loss:  0.010108952 d loss:  9.508235 w_loss:  0.009057431
epoch: 8
gen loss:  0.011747823 d loss:  9.424486 w_loss:  0.0132582635
epoch: 9
gen loss:  0.014804979 d loss:  9.233708 w_loss:  0.01729004
epoch: 10
gen loss:  0.015818799 d loss:  9.091016 w_loss:  0.02111205
epoch: 11
gen loss:  0.018488916 d loss:  8.883389 w_loss:  0.023414794
epoch: 12
gen loss:  0.020306852 d loss:  8.629396 w_loss:  0.03098574
epoch: 13
gen loss:  0.022511553 d loss:  

In [8]:
kwargs_opt = {
    "initial" : 5,
    "inn" : 10,
    "iterations" : 20,
    "optimizer_epochs" : 5000,
    "gan" : gan
}


optimizer = Optimize(**kwargs_opt)

In [9]:
nLatent = ndims_latent_input

t0 = time.time()
print('training_data', training_data.shape)
start_from = 100
inn = training_data[start_from,:(gan.nsteps-1)*nPOD].reshape(1, (gan.nsteps - 1) * nLatent)
print('inn',inn.shape)
npredictions = 20
#nLatent = 5 #latent_input_size = 5
nepochs_optimiser = 5000
initial_comp = training_data[start_from,:(nsteps-1)*nPOD].reshape((nsteps - 1), nLatent)
flds = optimizer.timesteps(initial_comp, inn, npredictions)
print('flds',flds.shape)

# rescale 
flds = scaling.inverse_transform(flds).T
print('shape of predictions before writing to file', flds.shape)

np.savetxt('optimised_prediction_from_GAN.csv', flds, delimiter=',')  #save gan input if continuing training after job ends

t_optimise = time.time() - t0

f = open('log.txt',"a")
f.write('Time taken to optimise: %s \n' % str(t_optimise)  )
f.close()

import sys
print('python',sys.version)
print('numpy', np.__version__)
print('tf', tf.__version__)

training_data (200, 25)
inn (1, 20)
Time step: 	 0
Optimizer epoch: 	 0
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Optimizer epoch: 	 100
Optimizer epoch: 	 200
Optimizer epoch: 	 300
Optimizer epoch: 	 400
Optimizer epoch: 	 500
Optimizer epoch: 	 600
Optimizer epoch: 	 700
Optimizer epoch: 	 800
Optimizer epoch: 	 900
Optimizer epoch: 	 1000
Optimizer epoch: 	 1100
Optimizer epoch: 	 1200
Optimizer epoch: 	 1300
Optimizer epoch: 	 1400
O

Optimizer epoch: 	 4900
Time step: 	 6
Optimizer epoch: 	 0
Optimizer epoch: 	 100
Optimizer epoch: 	 200
Optimizer epoch: 	 300
Optimizer epoch: 	 400
Optimizer epoch: 	 500
Optimizer epoch: 	 600
Optimizer epoch: 	 700
Optimizer epoch: 	 800
Optimizer epoch: 	 900
Optimizer epoch: 	 1000
Optimizer epoch: 	 1100
Optimizer epoch: 	 1200
Optimizer epoch: 	 1300
Optimizer epoch: 	 1400
Optimizer epoch: 	 1500
Optimizer epoch: 	 1600
Optimizer epoch: 	 1700
Optimizer epoch: 	 1800
Optimizer epoch: 	 1900
Optimizer epoch: 	 2000
Optimizer epoch: 	 2100
Optimizer epoch: 	 2200
Optimizer epoch: 	 2300
Optimizer epoch: 	 2400
Optimizer epoch: 	 2500
Optimizer epoch: 	 2600
Optimizer epoch: 	 2700
Optimizer epoch: 	 2800
Optimizer epoch: 	 2900
Optimizer epoch: 	 3000
Optimizer epoch: 	 3100
Optimizer epoch: 	 3200
Optimizer epoch: 	 3300
Optimizer epoch: 	 3400
Optimizer epoch: 	 3500
Optimizer epoch: 	 3600
Optimizer epoch: 	 3700
Optimizer epoch: 	 3800
Optimizer epoch: 	 3900
Optimizer epo

Optimizer epoch: 	 4000
Optimizer epoch: 	 4100
Optimizer epoch: 	 4200
Optimizer epoch: 	 4300
Optimizer epoch: 	 4400
Optimizer epoch: 	 4500
Optimizer epoch: 	 4600
Optimizer epoch: 	 4700
Optimizer epoch: 	 4800
Optimizer epoch: 	 4900
Time step: 	 13
Optimizer epoch: 	 0
Optimizer epoch: 	 100
Optimizer epoch: 	 200
Optimizer epoch: 	 300
Optimizer epoch: 	 400
Optimizer epoch: 	 500
Optimizer epoch: 	 600
Optimizer epoch: 	 700
Optimizer epoch: 	 800
Optimizer epoch: 	 900
Optimizer epoch: 	 1000
Optimizer epoch: 	 1100
Optimizer epoch: 	 1200
Optimizer epoch: 	 1300
Optimizer epoch: 	 1400
Optimizer epoch: 	 1500
Optimizer epoch: 	 1600
Optimizer epoch: 	 1700
Optimizer epoch: 	 1800
Optimizer epoch: 	 1900
Optimizer epoch: 	 2000
Optimizer epoch: 	 2100
Optimizer epoch: 	 2200
Optimizer epoch: 	 2300
Optimizer epoch: 	 2400
Optimizer epoch: 	 2500
Optimizer epoch: 	 2600
Optimizer epoch: 	 2700
Optimizer epoch: 	 2800
Optimizer epoch: 	 2900
Optimizer epoch: 	 3000
Optimizer ep

Optimizer epoch: 	 3200
Optimizer epoch: 	 3300
Optimizer epoch: 	 3400
Optimizer epoch: 	 3500
Optimizer epoch: 	 3600
Optimizer epoch: 	 3700
Optimizer epoch: 	 3800
Optimizer epoch: 	 3900
Optimizer epoch: 	 4000
Optimizer epoch: 	 4100
Optimizer epoch: 	 4200
Optimizer epoch: 	 4300
Optimizer epoch: 	 4400
Optimizer epoch: 	 4500
Optimizer epoch: 	 4600
Optimizer epoch: 	 4700
Optimizer epoch: 	 4800
Optimizer epoch: 	 4900
flds (24, 5)
shape of predictions before writing to file (5, 24)
python 3.8.10 (default, May 19 2021, 18:05:58) 
[GCC 7.3.0]
numpy 1.20.2
tf 2.4.1


In [10]:
print(flds)

[[ 2.48437694e+00  2.48444370e+00  2.48108772e+00  2.47933176e+00
   2.47157381e+00  2.44318830e+00  2.43999882e+00  2.44120611e+00
   2.44094482e+00  2.44102235e+00  2.44114754e+00  2.44105810e+00
   2.44140379e+00  2.44470077e+00  2.44422283e+00  2.44438931e+00
   2.44434238e+00  2.44434047e+00  2.44434967e+00  2.44434216e+00
   2.44434313e+00  2.44432341e+00  2.44434021e+00  2.44434429e+00]
 [-4.79158543e-02  2.34611976e-01  4.19823169e-01  4.38214978e-01
   1.28019132e-01 -7.32401690e-02 -1.10376114e-01 -8.24833627e-02
  -8.86116108e-02 -8.77867357e-02 -8.74045890e-02 -8.92571116e-02
  -8.76765416e-02 -5.35871252e-02 -6.16143742e-02 -5.97064979e-02
  -6.01815672e-02 -6.02717543e-02 -6.01607699e-02 -6.02321657e-02
  -6.02344015e-02 -6.04288724e-02 -6.02915519e-02 -6.02142999e-02]
 [-3.99152924e-01 -3.49496850e-01 -1.58300247e-01  9.86271254e-02
  -7.25238469e-02  8.83169115e-03  1.90335943e-02  2.14992126e-02
   2.08961215e-02  2.03950305e-02  2.06031760e-02  1.98230314e-02
   1.957